In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 분야별 클러스터링

In [3]:
!pip install konlpy
!pip tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 37.8 MB/s 
ERROR: unknown command "tqdm"


In [4]:
import pandas as pd
import numpy as np

import konlpy
from tqdm import tqdm
from konlpy.tag import Okt

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import DBSCAN

In [223]:

def preprocessing_clustering(df, part):

  if part == 'int':
    df = df[df['분야'] == part]
  else:
    df = df[df['분야'] != part]
    


  df = df.drop_duplicates(['타이틀'], keep = 'first')
  # df.shape

  df['text50'] = np.nan
  df['text15'] = np.nan

  for index, row in df.iterrows():
    if len(str(df['본문'][index])) >= 50:
      df['text50'][index] = str(df['본문'][index])[:50]
    else:
      df['text50'][index] = str(df['본문'][index])

    if len(str(df['본문'][index])) >= 15:
      df['text15'][index] = str(df['본문'][index])[:15]
    else:
      df['text15'][index] = str(df['본문'][index])

  df = df.drop_duplicates(subset = 'text15', keep = 'first')

  regions = ['뉴스핌', '연예', '서울', '경기', '강원', '충청', '경상', '전라', '인천', '경남', '경북', '대구', '울산', '부산', '광주', '전남', '제주', '전북', '대전', '세종', '충남', '충북', '충청북도', '충청남도', '경상남도', '경상북도', '전라북도', '전라남도']


  drop_index_list = [] # 지워버릴 index를 담는 리스트
  for i, row in df.iterrows():
      temp_region = row['text15']
      # temp_region = row['본문'].split(".")[0]
      for region in regions:
        if region in temp_region: 
            drop_index_list.append(i)
  df = df.drop(drop_index_list) # 해당 index를 지우기
  # doc.shape


  okt = Okt() # 형태소 분석기 객체 생성
  noun_list = []
  for content in tqdm(df['text50']): 
      nouns = okt.nouns(content) # 명사만 추출하기, 결과값은 명사 리스트
      noun_list.append(nouns)

  df['nouns'] = noun_list


  # 명사리스트 비어있는 경우 삭제

  drop_index_list = [] # 지워버릴 index를 담는 리스트
  for i, row in df.iterrows():
      temp_nouns = row['nouns']
      if len(temp_nouns) == 0: # 만약 명사리스트가 비어 있다면
          drop_index_list.append(i) # 지울 index 추가
          
  df = df.drop(drop_index_list) # 해당 index를 지우기

  # index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
  df.index = range(len(df))

  if part == 'int':
    min_df = 6
    min_samples = 5
  elif df.shape[0] < 10000:
    min_df = 5
    min_samples = 5
  else:
    min_df = 6
    min_samples = 5


  # 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
  text = [" ".join(noun) for noun in df['nouns']]

  tfidf_vectorizer = TfidfVectorizer(min_df = min_df, ngram_range=(1,3))
  tfidf_vectorizer.fit(text)
  vector = tfidf_vectorizer.transform(text).toarray()


  vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
  model = DBSCAN(eps=0.7,min_samples=min_samples, metric = "cosine")
  # 거리 계산 식으로는 Cosine distance를 이용
  result = model.fit_predict(vector)

  
  df['result'] = result
  if part == 'int':
    df['result'] = df['result'] + 100


  print(f"\n {df['result'].unique()} \n")

  return df

In [224]:
def cluster_result(df):

  cluster_idx = {}
  cluster_nums = {}
  for cluster_num in set(df['result']):
      # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
      if(cluster_num == -1 or cluster_num == 0 or cluster_num == 99 or cluster_num == 100): 
          continue
      else:
          print("cluster num : {}".format(cluster_num))
        
          temp_df = df[df['result'] == cluster_num] # cluster num 별로 조회
          num = len(df.index[df['result'] == cluster_num].to_list())
          for title in temp_df['타이틀']:
              print(title)
          cluster_idx[cluster_num] = df.index[df['result'] == cluster_num].to_list()
          cluster_nums[cluster_num] = num

          print("docs_num : {}".format(num))

          print()


  print(cluster_idx)

  return cluster_nums

In [225]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/20220825_all_soc.xlsx") # , encoding = "CP949"
df.head()

,분야,타이틀,발행일자,링크,본문
0,pol,우린 우거지예요!(우리의 거대한 지구사랑),2022-08-26,https://news.nate.com/view/20220825n41233?mid=...,환경과 무관한 사람이 있을까요. 요즘 어딜 가나 제로웨이스트나 친환경 관련 이야기가...
1,pol,[청년 픽!] 대학생도 국민내일배움카드!,2022-08-26,https://news.nate.com/view/20220825n41232?mid=...,국민내일배움카드는 급격한 기술 발전 및 노동시장 변화에 대응하는 사회안전망 차원에서...
2,pol,[청년 픽!] 청년전세임대주택 살아 보니~,2022-08-26,https://news.nate.com/view/20220825n41231?mid=...,‘주거’는 인간이 생활하는데 기본적으로 갖춰져야 할 의·식·주 중 하나를 이루고 있...
3,pol,이런 게 바로 혁신제품이지~,2022-08-26,https://news.nate.com/view/20220825n41230?mid=...,"혁신제품 권역별 순회 전시회 참관기8월 24일, 부산 벡스코 1층 4F홀에서 혁신제..."
4,pol,"윤석열 대통령실 커피, 국민의힘 국회의원 연찬회 등장",2022-08-25,https://news.nate.com/view/20220825n29314?mid=...,ⓒ데일리안 홍금표 기자25일 오후 충남 천안 재능교육연수원에서 열린 국민의힘 202...


In [226]:
df.shape

(9766, 5)

In [227]:
part_name = ['all'] # , 'int'

news_df = pd.DataFrame()

for part in part_name:
  df_result = preprocessing_clustering(df, part)
  news_df = pd.concat([news_df,df_result])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 5537/5537 [00:20<00:00, 275.86it/s]



 [-1  0  1  2  3  4  5  7  6  8  9] 



In [228]:
news_df['result'].unique()

array([-1,  0,  1,  2,  3,  4,  5,  7,  6,  8,  9])

In [229]:
news_df.head(3)

,분야,타이틀,발행일자,링크,본문,text50,text15,nouns,result
0,pol,우린 우거지예요!(우리의 거대한 지구사랑),2022-08-26,https://news.nate.com/view/20220825n41233?mid=...,환경과 무관한 사람이 있을까요. 요즘 어딜 가나 제로웨이스트나 친환경 관련 이야기가...,환경과 무관한 사람이 있을까요. 요즘 어딜 가나 제로웨이스트나 친환경 관련 이야기가...,환경과 무관한 사람이 있을까,"[환경, 무관, 사람, 요즘, 가나, 제로, 웨, 이스트, 친환경, 관련, 이야기,...",-1
1,pol,[청년 픽!] 대학생도 국민내일배움카드!,2022-08-26,https://news.nate.com/view/20220825n41232?mid=...,국민내일배움카드는 급격한 기술 발전 및 노동시장 변화에 대응하는 사회안전망 차원에서...,국민내일배움카드는 급격한 기술 발전 및 노동시장 변화에 대응하는 사회안전망 차원에서...,국민내일배움카드는 급격한 기,"[국민, 배움, 카드, 기술, 발전, 및, 노동시장, 변화, 대응, 회안, 전망, ...",-1
2,pol,[청년 픽!] 청년전세임대주택 살아 보니~,2022-08-26,https://news.nate.com/view/20220825n41231?mid=...,‘주거’는 인간이 생활하는데 기본적으로 갖춰져야 할 의·식·주 중 하나를 이루고 있...,‘주거’는 인간이 생활하는데 기본적으로 갖춰져야 할 의·식·주 중 하나를 이루고 있...,‘주거’는 인간이 생활하는데,"[주거, 인간, 생활, 기본, 의, 식, 주, 중, 하나, 만큼]",-1


In [230]:
cluster_nums = cluster_result(news_df)

cluster num : 1
정권 초 이례적 고강도 감찰…권력 충돌? 尹 자신감?
대통령실 고강도 감찰 중…정상화? 권력 충돌?
대통령실, 정권 초 이례적 고강도 감찰…정상화? 권력 충돌? 
정권 초 이례적 고강도 감찰…정상화? 권력 충돌?
docs_num : 4

cluster num : 2
美 크리튼브링크 차관보 방한…"한국과 긴밀히 협의"
크리튼브링크 美동아태차관보 방한…한미관계 현안 논의(종합)
크리튼브링크 미 동아태차관보 방한
美동아태차관보 방한…인플레감축법 논의될듯
美 동아태 차관보 오늘 방한…한미동맹 강화 등 논의
docs_num : 5

cluster num : 3
영광군, 3년만에 열리는 '불갑산상사화축제' 준비 만전
담양군 무정면에 18홀 골프장 들어선다
담양군 2023국제 슬로시티 재인증 준비 본격화
나주시, 2년 만에 환경미화원 10명 공개 채용
해남군 '농식품 기후변화대응센터' 조성 예타 통과
docs_num : 5

cluster num : 4
웜비어 부모, 탈북민에 장학금…"北 정권에 강력한 메시지"
웜비어 부모, 탈북민에 장학금…"北정권에 강력 메시지 될것"
웜비어 부모, 탈북민에 장학금 수여…"北에 강력한 메시지 되길"
웜비어 부모, 탈북민에 장학금
윔비어 부모, 탈북민에 장학금…"北정권에 강력한 메시지 되길"
웜비어 부모, 탈북민에 장학금…수혜자는 美대학원생 이서현씨
docs_num : 6

cluster num : 5
해군 진기사 실전 같은 훈련
드론 테러 '꼼짝마'…하늘 경계 이상 없다
BNK경남은행, 창원특례시에 창원사랑상품권 1억원 기탁
경남은행, 창원특례시에 '창원사랑상품권 1억' 기탁
docs_num : 4

cluster num : 6
추석 차례상 비용 31만원…대형마트보다 시장이 평균 9만원 저렴
시금치가 또 '시金치' 됐다…추석 때마다 가격 급등하는 이유
팍팍 뛰는 물가…"올추석 차례상 비용 30만원 넘을 것"
고물가에 추석 차례상 비용 31만원…'시장이 25% 싸다'
aT, 이천비축기지 민관군경 합동 훈련으로 

In [231]:
def top10(news_df, cluster_nums):
  result_clus = sorted(cluster_nums.items(), key = lambda x:x[1], reverse = True)[:10]
  print(result_clus)

  fin_df = pd.DataFrame()
  clus_group = []
  for clus in result_clus:
    clus_group.append(clus[0])
    print(f"{clus[0]}---------------------------------------")
    temp_df = news_df[news_df['result'] == clus[0]] # cluster num 별로 조회
    print(temp_df['본문'])

    fin_df = pd.concat([fin_df, temp_df])
  
  return fin_df, clus_group

In [232]:
fin_df, clus_group = top10(news_df, cluster_nums)
fin_df['result'].unique()

[(7, 11), (4, 6), (6, 6), (2, 5), (3, 5), (8, 5), (1, 4), (5, 4), (9, 4)]
7---------------------------------------
1345    친환경 소비 관련 할인·적립사용액 일부 공익단체에 기부ESG(환경·사회·지배구조)에...
2182    3년 연속 발간하며 ESG 경영 강화[서울=뉴스핌] 정승원 기자 = 넥센타이어는 지...
2244    금호타이어 '에어본 타이어', 독일 레드닷 디자인 어워드 2022 본상금호타이어 U...
2334     넥센타이어는 지속가능경영 성과를 담은 '2021·22 환경·사회·지배구조(ESG)...
2998    넥센타이어가 지속가능경영 성과를 담은 ESG 보고서를 발간했다고 25일 밝혔다.이번...
3335    넥센타이어 ESG 리포트./제공=넥센타이어아시아투데이 정문경 기자 = 넥센타이어가 ...
3595    SHE 기획팀 신설·ESG 운영 조직 개선…ESG 경영체제 강화프라임경제 넥센타이어...
3678    2년 연속 발간, ESG 경영 체제 강화넥센타이어가 25일 지속가능경영 성과를 담은...
3956    ▲ 넥센타이어는 지속가능경영 성과를 담은 2021/22 ESG(환경·사회·지배구조)...
4236    SHE기획팀 신설…ESG경영 체제 강화넥센타이어 2021/22 ESG 리포트. [넥...
4481    상업용지 용도변경 협상 난항금호타이어 광주공장[헤럴드경제(광주)=서인주 기자] 금호...
Name: 본문, dtype: object
4---------------------------------------
711     북한에 억류됐다 숨진 미국인 대학생 오토 웜비어의 부모가 미국에 정착한 탈북민에게 ...
1160    '39호실' 출신 탈북민 리정호 딸에게 수여키로납북·억류 피해자 韓기자회견 참석했던...
4004    오토 웜비어의 부친인 프레드 웜비어 /사진=연합뉴스북한에 억류됐다 숨진 미국인 대학...
4827    

array([7, 4, 6, 2, 3, 8, 1, 5, 9])

#### 클러스터링 된 집합에서 대표 뉴스 선정
- [] 랜덤

In [220]:
fin_df.head(3)

,분야,타이틀,발행일자,링크,본문,text50,text15,nouns,result
1345,eco,착한 소비로 혜택 받아볼까…'ESG 카드' 주목,2022-08-25,https://news.nate.com/view/20220825n22641?mid=...,친환경 소비 관련 할인·적립사용액 일부 공익단체에 기부ESG(환경·사회·지배구조)에...,친환경 소비 관련 할인·적립사용액 일부 공익단체에 기부ESG(환경·사회·지배구조)에...,친환경 소비 관련 할인·적립,"[친환경, 소비, 관련, 할인, 적립, 용액, 일부, 공익단체, 기부, 환경, 사회...",6
2182,eco,"넥센타이어, 지속가능경영 성과 담은 ESG 리포트 발간",2022-08-25,https://news.nate.com/view/20220825n26520?mid=...,3년 연속 발간하며 ESG 경영 강화[서울=뉴스핌] 정승원 기자 = 넥센타이어는 지...,3년 연속 발간하며 ESG 경영 강화[서울=뉴스핌] 정승원 기자 = 넥센타이어는 지...,3년 연속 발간하며 ESG,"[연속, 발간, 경영, 강화, 서울, 뉴스, 정승원, 기자, 넥센, 타이어, 가능, 경]",6
2244,eco,"금호타이어, UAM용 시스템 타이어로 獨 디자인상 본상 '쾌거'",2022-08-25,https://news.nate.com/view/20220825n25799?mid=...,"금호타이어 '에어본 타이어', 독일 레드닷 디자인 어워드 2022 본상금호타이어 U...","금호타이어 '에어본 타이어', 독일 레드닷 디자인 어워드 2022 본상금호타이어 U...",금호타이어 '에어본 타이어',"[금호, 타이어, 에어, 타이어, 독일, 레드, 닷, 디자인, 어워드, 본상, 금호...",6


In [233]:
## 클러스터별 랜덤으로 1개 추출
import random

def final_dataset(fin_df, clus_group):

  # fin_df 인덱스 재설정
  fin_df.index = range(len(fin_df))

  # final_news = pd.DataFrame()
  idx = []
  for clus in clus_group:
    temp_idx = fin_df[fin_df['result'] == clus].index
    idx.append(temp_idx[random.randrange(0, len(temp_idx))])
    # print(temp_idx)

  # print(idx)
  final_news = fin_df.iloc[idx, :5]

  return final_news



In [234]:
final_news = final_dataset(fin_df, clus_group)
final_news

,분야,타이틀,발행일자,링크,본문
2,eco,"금호타이어, UAM용 시스템 타이어로 獨 디자인상 본상 '쾌거'",2022-08-25,https://news.nate.com/view/20220825n25799?mid=...,"금호타이어 '에어본 타이어', 독일 레드닷 디자인 어워드 2022 본상금호타이어 U..."
13,eco,"웜비어 부모, 탈북민에 장학금 수여…""北에 강력한 메시지 되길""",2022-08-25,https://news.nate.com/view/20220825n07537?mid=...,오토 웜비어의 부친인 프레드 웜비어 /사진=연합뉴스북한에 억류됐다 숨진 미국인 대학...
17,eco,추석 차례상 비용 31만원…대형마트보다 시장이 평균 9만원 저렴,2022-08-25,https://news.nate.com/view/20220825n32644?mid=...,농수산식품유통公 명절 성수품 조사 고물가 행진으로 올해 추석 차례상 차림비용은 지난...
25,pol,크리튼브링크 미 동아태차관보 방한,2022-08-25,https://news.nate.com/view/20220825n35444?mid=...,(영종도=연합뉴스) 김주성 기자 = 대니얼 크리튼브링크 미국 국무부 동아시아태평양 ...
28,pol,"영광군, 3년만에 열리는 '불갑산상사화축제' 준비 만전",2022-08-25,https://news.nate.com/view/20220825n24625?mid=...,"'상사화 붉은물결, 청춘의 사랑을 꽃피우다' 주제[영광=뉴스핌] 조은정 기자 = 전..."
36,eco,"롯데월드타워, '서울시 좋은빛상' 최우수상 수상",2022-08-25,https://news.nate.com/view/20220825n04731?mid=...,롯데월드타워는 지난 4월 지구의 날을 맞아 환경파괴 위험성과 동식물 보호 메시지를...
41,pol,정권 초 이례적 고강도 감찰…정상화? 권력 충돌?,2022-08-25,https://news.nate.com/view/20220825n16072?mid=...,[앵커]지난 100일간 손발을 맞춘 대통령실이 최근 전 직원을 대상으로 강도 높은 ...
44,eco,"BNK경남은행, 창원특례시에 창원사랑상품권 1억원 기탁",2022-08-25,https://news.nate.com/view/20220825n29894?mid=...,창원지역 소외계층 1000세대 지원25일 창원시 성산구 창원시청에서 진행된 창원사랑...
49,eco,"포니정재단, 고려대와 '포니정 아카이브' 연구 시동",2022-08-25,https://news.nate.com/view/20220825n10946?mid=...,[이투데이/정용욱 기자]고려대 민족문화연구원 관계자들이 24일 출범식 직후 기념촬영...


In [172]:
final_news.to_excel("/content/drive/MyDrive/Colab Notebooks/clustered_0826.xlsx", encoding = "utf-8")